# Learning the Best Diabetes Medication

Here we implement the Bayesian belief model from Chapter 4 to find the best Diabetes medication. The following inputs are needed to fully specify the statistical model:

1. `S0`: For every drug, we maintain a *belief* about its A1C reduction. The beliefs are modelled as a set of normal distributions (i.e., two parameters per drug) that evolve as we make observations. The initial belief is specified with a mean and standard deviation derived from, e.g., the efficacy of each drug over an entire population (many individuals).
2. `mu_truth`: When simulating the model, the true (but unknown) value for the A1C reduction of every drug must be simulated as well. We do that by directly passing a `scipy.stats.uniform` object, from which the model can draw samples. Caution: The two arguments of `scipy.stats.uniform`, `loc` and `scale` are not the upper and lower bound, but the lower and upper bounds will be `[loc, loc+scale]`. It is also possible to select fixed values for `mu_truth`. In this case, select a uniform distribution where the lower and upper bound are equal. 
3. `sigma_W`: This is the standard deviation of an observation. Observations are sampled from a normal distribution with mean `mu_truth` and standard deviation `sigma_W`.

We first create a model where `mu_truth` is fixed, that means we have only one random process (observational uncertainty).

In [ ]:
import scipy.stats
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import MedicalDecisionDiabetesModel as mddm
import MedicalDecisionDiabetesPolicies as mddp
import BaseClasses.Util as util

S0 = {
    "M":    [0.32, 0.12],
    "Sens": [0.28, 0.09],
    "Secr": [0.3, 0.17],
    "AGI":  [0.26, 0.15],
    "PA":   [0.21, 0.11],
}

mu_truth = {
    "M":    scipy.stats.uniform(loc=0.3, scale=0.0),
    "Sens": scipy.stats.uniform(loc=0.2, scale=0.0),
    "Secr": scipy.stats.uniform(loc=0.4, scale=0.0),
    "AGI":  scipy.stats.uniform(loc=0.33, scale=0.0),
    "PA":   scipy.stats.uniform(loc=0.35, scale=0.0),
}

model = mddm.MedicalDecisionDiabetesModel(S0=S0, mu_truth=mu_truth, sigma_W=0.05, T=20)

Next, we create an upper confidence bound policy and run it for 1000 iterations. In each iteration, we have 20 observations/trials. The objective is to maximize the total level of A1C reduction with these 20 trials. 

In [ ]:
policy = mddp.UCB(model, theta=1)
policy.run_policy(n_iterations=1000)

We have a closer look at one sample run. The state variable for every drug are given as triples. We create one column for mean, standard deviation, and $N_x$. 

In [ ]:
for drug in S0.keys():
    policy.results[drug + "_mu"] = policy.results[drug].apply(lambda x: x[0])
    policy.results[drug + "_sigma"] = 1.0/policy.results[drug].apply(lambda x: np.sqrt(x[1]))
    policy.results[drug + "_N"] = policy.results[drug].apply(lambda x: x[2])
    policy.results[drug + "_chosen"] = policy.results.groupby("N")[drug + "_N"].diff()

Now we plot a random iteration of 20 trials. We plot
- the current belief $\mu^n_x$ for every drug
- the current uncertainty in the belief $\sigma^n_x$ as errorbars

Only when a drug is chosen, $\mu^n_x$ and $\sigma^n_x$ will change. $\sigma^n_x$ will be monotonically decreasing (we are getting more certain the more often we try a drug).

In [ ]:
sample_paths = np.random.choice(1000, size=1, replace=False)
df = policy.results.loc[policy.results.N.isin(sample_paths), :]

long_df = df.melt(id_vars=["t","N"], value_vars=["M_mu", "Sens_mu", "Secr_mu", "AGI_mu", "PA_mu"], value_name="mu", var_name="drug")
long_df["sigma"] = df[["M_sigma", "Sens_sigma", "Secr_sigma", "AGI_sigma", "PA_sigma"]].unstack().values
long_df["chosen"] = df[["M_chosen", "Sens_chosen", "Secr_chosen", "AGI_chosen", "PA_chosen"]].unstack().values

px.line(data_frame=long_df, x="t", y="mu", color="drug", error_y="sigma", facet_row="N", hover_data="chosen", markers=True)

## Exercise 1
Perform a grid search for the UCB policy with values $\theta=0.0,0.2,\dots,2.0$ and plot the performance for each value of $\theta$. What do you learn from this plot?

---

## Exercise 2
In this exercise we investigate the interval estimation policy. We are going to evaluate it for three different sets of thruths. For each of the three cases run a grid search for $\theta=0.0,0.2,\dots,2.0$ with 10000 iterations and plot the average performance against the value of $\theta$.

1. Use the values for `S0` and `mu_truth` just as given at the beginning of the notebook (you can even reuse the model object). 
2. Let $\mu_x^0$ be your initial belief about the performance of drug $x$. Use $\mu_x^0$ as given in `S0` above but simulate the truth by taking a sample of a uniform distribution on the interval $[0.5\mu_x^0, 1.5\mu_x^0]$. This is an example of having a prior distribution of belief (in this case, that is normally distributed) but sampling the truth from a different distribution (that is uniformly distributed around the mean).
3. Set `S0` such that the prior is $\mu_x^0=0.3$ for all five drugs $x$ with initial standard deviation $\sigma_x^0=0.1$. Sample `mu_truth` for all five drugs uniformly from the interval $[0.15,0.45]$.

What conclusions can you draw from each of the plots?